First, import and load the image.

In [ ]:
# Imports
from PIL import Image
import colorsys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm

# Import the image
img = Image.open("image324x324inside.jpg")
# img = Image.open("image324x324outside.jpg")


# Define constants

In [49]:
def apply_nxn_kernel(img, kernel, n):
    pad = n-1 // 2

    new_img = Image.new("RGB", (img.width - pad*2, img.height - pad*2))

    for x in range(pad, img.width-pad):
        for y in range(pad, img.height-pad):
            pixels = np.zeros((n, n, 3))
            for i in range(n):
                for j in range(n):
                    pixels[i, j] = img.getpixel((x - pad + i, y - pad + j))
            
            r, g, b = kernel(pixels)

            new_img.putpixel((x - pad, y - pad), (r, g, b))

    return new_img

def blur(pixels):
    n = 5

    blur = np.array([
        [1/256, 4/256,  6/256,  4/256,  1/256],
        [4/256, 16/256, 24/256, 16/256, 4/256],
        [6/256, 24/256, 36/256, 24/256, 6/256],
        [4/256, 16/256, 24/256, 16/256, 4/256],
        [1/256, 4/256,  6/256,  4/256,  1/256],
    ])

    r, g, b = 0, 0, 0
    for i in range(n):
        for j in range(n):
            r += pixels[i, j][0] * blur[i, j]
            g += pixels[i, j][1] * blur[i, j]
            b += pixels[i, j][2] * blur[i, j]

    r = int(min(max(r, 0), 255))
    g = int(min(max(g, 0), 255))
    b = int(min(max(b, 0), 255))

    return (r, g, b)

def edge(pixels):
    n = 3

    edge = np.array([
        [-1, -1, -1],
        [-1,  8, -1],
        [-1, -1, -1],
    ])

    r, g, b = 0, 0, 0
    for i in range(n):
        for j in range(n):
            r += pixels[i, j][0] * edge[i, j]
            g += pixels[i, j][1] * edge[i, j]
            b += pixels[i, j][2] * edge[i, j]

    r = int(min(max(r, 0), 255))
    g = int(min(max(g, 0), 255))
    b = int(min(max(b, 0), 255))

    return (r, g, b)

def threshold(pixels):
    t = 30

    r = pixels[0, 0][0]
    g = pixels[0, 0][1]
    b = pixels[0, 0][2]

    v = 255 if (r + g + b) / 3 > t else 0

    return (v, v, v)


img_blur = apply_nxn_kernel(img, blur, 5)
img_edge = apply_nxn_kernel(img_blur, edge, 3)
img_threshold = apply_nxn_kernel(img_edge, threshold, 1)

img.show()
img_blur.show()
img_edge.show()
img_threshold.show()